# Geospatial Machine Learning: Classifying Land Use Based on Environmental and Economic Features

# 1. Introduction

In this project, the dataset we decided to analyze is the **Geospatial Environmental and Socioeconomic Data**. We believe we can extract valuable insights by developing a **classification model** that can **predict the primary land cover type** of a geographic area based on spatial indicators such as population density, GDP, temperature, and proximity to roads and cities. To accomplish this, we will use **machine learning algorithms** to investigate whether there are **patterns linking socioeconomic and environmental conditions** to different types of **land use**.


# 2. Description of the Dataset

## Geospatial Environmental and Socioeconomic Data  
**by Cathetorres (2020) – Kaggle**

This dataset is a comprehensive compilation of global **geospatial vector and raster data**, integrating diverse environmental and socioeconomic indicators. Collected from multiple authoritative sources, it provides detailed spatial layers useful for analysis in sustainability, development, and environmental monitoring.

The dataset is structured into **12 thematic folders**, each covering a distinct data category:

1. **Cities and Towns** – Geospatial data for urban areas in point and polygon formats.  
2. **Roads and Railroads** – Global transportation infrastructure networks.  
3. **Airports and Ports** – Locations of major air and sea transportation hubs.  
4. **Power Plants** – Spatial distribution of energy production facilities.  
5. **Gridded Population (2015)** – High-resolution (250 m) global population estimates.  
6. **Gridded GDP and HDI (1990–2015)** – Socioeconomic data showing economic productivity and development trends over time.  
7. **Land Cover (2015)** – Classification of global land cover types based on satellite data.  
8. **Tree Cover Loss by Dominant Driver** – Causes of deforestation disaggregated by primary driving factors.  
9. **Carbon Accumulation Potential** – Estimates of carbon sequestration from natural forest regrowth in forests and savannas.  
10. **Solar Energy Potential** – Spatial potential for solar power generation across the globe.  
11. **Air Temperature** – Global surface temperature data for climate studies.  
12. **Global Cattle Distribution (2010)** – Spatial density of cattle populations with 5-arcminute resolution.

## Dataset Folders Used

For this project, we utilize selected folders from the **Geospatial Environmental and Socioeconomic Data** collection to build a classification model that predicts the **primary land cover type** of a geographic area based on environmental and socioeconomic indicators. Below is a breakdown of the relevant folders used in the analysis:

### Target Variable

- **7. Gridded Land Cover 2015**  
  This dataset provides the land cover classification for each 250 m grid cell. It serves as the **label** for our machine learning model, representing categories such as forest, cropland, urban area, grassland, water, etc.

### Input Features

#### Human and Economic Indicators
- **5. Gridded Population 2015 (250 m)**  
  Provides population density data per grid cell, which may correlate with urban or peri-urban land use.

- **6. Gridded GDP and HDI (1990–2015)**  
  Economic indicators used to understand development intensity, which may affect land use patterns.

#### Environmental Variables

- **11. Air Temperature**  
  Climatic data used to correlate temperature patterns with land cover types like tundra, desert, or tropical forest.

- **12. Global Cattle Distribution (2010)**  
  Used as a proxy for agricultural activity, particularly pastoral land use.


# 3. Importing Files and Python Libraries

Here are the libraries and modules that will be used in this notebook:

In [1]:
import rasterio
import numpy as np
import pandas as pd
import xarray as xr

from rasterio.enums import Resampling
from skimage.transform import resize
from rasterio.transform import rowcol

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

### Importing the Files

In [2]:
raster_path = 'Goode_FinalClassification_19_05pcnt_prj.tif'

# 4. Data Precessing and Cleaning

In [8]:
# Dictionary of provinces with approximate central coordinates
philippine_provinces = {
    "Manila (NCR)": (14.6, 120.98),
    "Benguet": (16.42, 120.60),
    "Isabela": (17.07, 121.85),
    "Palawan": (9.83, 118.73),
    "Cebu": (10.32, 123.90),
    "Leyte": (10.85, 124.95),
    "Davao del Sur": (6.80, 125.45),
    "Zamboanga del Sur": (7.83, 123.45),
    "Agusan del Norte": (9.00, 125.60),
    "Sulu": (6.02, 121.00)
}


In [3]:
def get_values_in_bounds(data, transform, lat_min, lat_max, lon_min, lon_max):
    values = []
    for lat in np.arange(lat_min, lat_max, 0.1):  # step = resolution
        for lon in np.arange(lon_min, lon_max, 0.1):
            row, col = rasterio.transform.rowcol(transform, lon, lat)
            if 0 <= row < data.shape[0] and 0 <= col < data.shape[1]:
                val = data[row, col]
                values.append(val)
    return np.array(values)

**Loading .tif raster layers**

In [ ]:
# Open and process raster
with rasterio.open(raster_path) as src:
    data = src.read(1)
    transform = src.transform
    nodata = src.nodata

    print("\nTree Cover Loss Raster Values by Province (2001–2019):\n")
    for province, (lat, lon) in philippine_provinces.items():
        try:
            row, col = rowcol(transform, lon, lat)
            value = data[row, col]
            print(f"{province:<25}: {value}")
        except IndexError:
            print(f"{province:<25}: Coordinates outside raster bounds")


Tree Cover Loss Raster Values by Province (2001–2019):

Manila (NCR)             : 0.0
Benguet                  : 5.0
Isabela                  : 3.0
Palawan                  : 2.0
Cebu                     : 0.0
Leyte                    : 3.0
Davao del Sur            : 0.0
Zamboanga del Sur        : 5.0
Agusan del Norte         : 5.0
Sulu                     : 3.0


In [6]:
# Open raster
with rasterio.open(raster_path) as src:
    data = src.read(1)
    transform = src.transform
    nodata = src.nodata

    print("\nTree Cover Loss Raster Value Counts by Island Group (2001–2019):\n")

    island_bounds = {
    "Luzon":    (12.0, 21.0, 118.0, 124.0),
    "Visayas":  (9.5, 12.0, 122.0, 126.0),
    "Mindanao": (4.5, 9.5, 124.0, 127.0)
}

    for region, (lat_min, lat_max, lon_min, lon_max) in island_bounds.items():
        vals = get_values_in_bounds(data, transform, lat_min, lat_max, lon_min, lon_max)

        # Include everything: 0, 1–5, and potentially nodata
        unique, counts = np.unique(vals, return_counts=True)

        print(f"{region} ({len(vals)} total pixels):")
        for u, c in zip(unique, counts):
            print(f"  Value {u}: {c} pixels")
        print()



Tree Cover Loss Raster Value Counts by Island Group (2001–2019):

Luzon (5400 total pixels):
  Value -1.7e+308: 3961 pixels
  Value 0.0: 503 pixels
  Value 1.0: 105 pixels
  Value 2.0: 639 pixels
  Value 3.0: 122 pixels
  Value 5.0: 70 pixels

Visayas (1000 total pixels):
  Value -1.7e+308: 372 pixels
  Value 0.0: 241 pixels
  Value 1.0: 13 pixels
  Value 2.0: 270 pixels
  Value 3.0: 79 pixels
  Value 5.0: 25 pixels

Mindanao (1500 total pixels):
  Value -1.7e+308: 771 pixels
  Value 0.0: 105 pixels
  Value 1.0: 130 pixels
  Value 2.0: 337 pixels
  Value 3.0: 126 pixels
  Value 5.0: 31 pixels



**Loading .nc**

In [ ]:
print(data[0:5, 0:5])  # Print a 5×5 subset
import numpy as np




[[-1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308]
 [-1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308]
 [-1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308]
 [-1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308]
 [-1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308 -1.7e+308]]
Unique values in raster: [-1.7e+308  0.0e+000  1.0e+000  2.0e+000  3.0e+000  4.0e+000  5.0e+000]


**Array Shapes**

As seen, each has different shapes.

In [16]:
unique_vals = np.unique(data)
print("Unique values in raster:", unique_vals)

Unique values in raster: [-1.7e+308  0.0e+000  1.0e+000  2.0e+000  3.0e+000  4.0e+000  5.0e+000]


**Resampling**

New Array Shape

# 5. Exploratory Data Analysis

## 1. What is the distribution of household sizes (PUFHHSIZE)?

In [ ]:
## 2. What is the distribution of age (PUFC05_AGE)?

## 3. What is the distribution of regions (PUFREG)?

## 4. What is the distribution of provinces (PUFPRV)?

## 5. What is the ratio of males vs. females (PUFC04_SEX)?

## 6. What are the most common education levels (PUFC07_GRADE)?

## 7. What are the most common occupations (PUFC14_PROCC)?

## 8. What are the most common industries (PUFC16_PKB)?

## 9. How do daily working hours differ by occupation? (PUFC14_PROCC, PUFC18_PNWHRS)?

In [ ]:
## 10. What are the top 5 occupations (PUFC07_GRADE) per nature of employment (PUFC14_PROCC)?

# Resampling

# Scaling

# Splitting the Dataset into training, testing, and validation

# Machine Learning

## Initial Model Training

## Error Analysis

## Improving Model Performance

# Insights and Conclusion

# References